In [1]:
## Bring in mods and write ftns for analysis
import pandas as pd, numpy as np, os, glob
from Bio import SeqIO

def dotfill(r,a,fill='.'):
    return ''.join([fill for i in range(abs(len(a)-len(r)))])

def scorealign(r,a,fill='.',verbose=False):
    score = 0
    ns = []
    for i in range(len(r))[1:-1]:
        temp = r[:i] + dotfill(r,a) + r[i:]
        c = 0
        for j,n in enumerate(a):
            if n == temp[j]:
                c = c + 1
        if c > score:
            ns = temp
            score = c
            if verbose:
                print(ns,c)
    if len(ns) == 0:
        #print('Error empty sequence\n%s\n%s'%(r,a))
        ns = r[0]+dotfill(r,a)+r[-1]
    return ns

def calcpos(seq,start,end,pad=0,fill='.'):
    c = 0;
    pos = np.arange(start,end+pad)
    poss= []
    if fill in seq:
        for n in seq:
            if n != fill:
                poss.append(pos[c])
                c = c + 1
            else:
                poss.append(pos[c])
    else:
        poss = pos
    return poss

def realign(ref,alt,verbose=False):
    if len(alt) > len(ref):
        newref = scorealign(ref,alt,verbose=verbose)
        newalt = alt
    elif len(ref) > len(alt):
        newalt = scorealign(alt,ref,verbose=verbose)
        newref = ref
    else:
        assert len(alt) == len(ref)
        newref = ref
        newalt = alt
    return newref,newalt

rdict = dict(zip(['three_prime_UTR',
                  'five_prime_UTR',
                  'CDS'],
                 [3,5,0]))

def gvtodf(gv,refseqdf,gffdf,rs='REF',fill='.',rdict=rdict,verbose=False):

    refs = []
    alts = []
    poss = []
    for i,j in gv.iterrows():
    
        ref = j.Ref
        alt = j.Alt.split('.')[int(j.Sample)]
        start = j.Pos
        end = start + len(ref)
    
        newref,newalt = realign(ref,alt)
        newpos = calcpos(newref,start,end)
    
        refs.append(newref)
        alts.append(newalt)
        poss.append(newpos)
    
    pos = np.concatenate(poss)
    ref = [n for n in ''.join(refs)]
    alt = [n for n in ''.join(alts)]

    df = pd.DataFrame([pos,ref,alt],
                      index=['Pos','Ref','Alt']).T
    df['Isvar'] = 1

    check = refseqdf[(refseqdf.Pos.isin(df.Pos))]
    failed = []
    for i,j in check.iterrows():
        
        tocheck = [s for s in 
                   df[(df.Pos==j.Pos)].Ref.tolist() 
                   if (s != fill)]
        if len(tocheck) > 1: ## Should be one base
            failed.append(j.Pos)
        elif not tocheck[0] == j[rs]:
            failed.append(j.Pos)
            
    if (len(failed) > 0) & verbose:
        print('Reference alignment failed')
    
    dfadd = refseqdf[~(refseqdf.Pos.isin(df.Pos))].copy()
    dfadd['Ref'] = dfadd[rs]
    dfadd['Alt'] = dfadd[rs]
    dfadd['Isvar'] = 0
    dfadd.drop(rs,axis=1,inplace=True)

    res = pd.concat([dfadd,df]).sort_values('Pos')
    #res['Strand'] = -1 if gffdf.Strand.min() == '-' else 1
    res['Strand'] = gffdf.Strand.min()
    res['Type'] = -1
    res['Phase'] = -1

    for i,j in gffdf[(gffdf.Type!='gene')].iterrows():
    
        nb = np.arange(j.Start,j.End+1)
        res.loc[(res.Pos.isin(nb)),'Type'] = j.Type
    
        if j.Type == 'CDS':
            res.loc[(res.Pos.isin(nb)),'Phase'] = int(j.Phase)
    
    assert -1 not in res[(res.Type=='CDS')].Phase.tolist()
    
    res.Type.replace(rdict,inplace=True)

    return res,failed

In [2]:
chrommap_path = '../../GENOTYPE/H99_chrommap.csv'
chrommap = pd.read_csv(chrommap_path)
chrommap.head()

,Chrom,Seqid,Length,Cumpos,Midpts
0,1,CP003820.1,2291499,0,1145749.5
1,2,CP003821.1,1621675,2291499,3102336.5
2,3,CP003822.1,1575141,3913174,4700744.5
3,4,CP003823.1,1084805,5488315,6030717.5
4,5,CP003824.1,1814975,6573120,7480607.5


In [3]:
#geno_path = '../../GENOTYPE/Bt22xFtc555-1_genotypes.csv.gz'
geno_path = '../../GENOTYPE/Bt22xFtc555-1_QTL_genotypes.csv.gz'
genos = pd.read_csv(geno_path,index_col=0)
genos = genos.merge(chrommap[['Chrom','Seqid']])

genos.head()

,Seqid,Pos,Qual,Callrate,Nallele,Alleles,Maxlen,Minlen,Depth,Type,...,PMY2567,PMY2868,PMY2659,PMY2879,PMY2882,PMY2887,Chrom,MAF,Mdepth,MAR
0,CP003824.1,1015010,332131.0,1.0,1,AAGTC.AAGTT,5,5,10745,snp,...,1.0,1.0,1.0,1.0,1,1,5,1.0,34.413681,0.951384
1,CP003824.1,1015035,333883.0,1.0,1,C.T,1,1,10643,snp,...,1.0,1.0,1.0,1.0,1,1,5,1.0,34.508143,0.955241
2,CP003824.1,1015148,295782.0,1.0,1,TAACA.TAGCA,5,5,9297,snp,...,1.0,1.0,1.0,1.0,1,1,5,1.0,29.811075,0.954066
3,CP003824.1,1015178,310092.0,1.0,1,CGAGGAAC.CGAAGAAC,8,8,9852,snp,...,1.0,1.0,1.0,1.0,1,1,5,1.0,31.485342,0.956476
4,CP003824.1,1015276,241338.0,1.0,1,AGGAGAAGG.AGGAAAAGG,9,9,8448,snp,...,1.0,1.0,1.0,1.0,1,1,5,1.0,25.869707,0.947224


In [4]:
genos[(genos.Pos==761938)]

,Seqid,Pos,Qual,Callrate,Nallele,Alleles,Maxlen,Minlen,Depth,Type,...,PMY2567,PMY2868,PMY2659,PMY2879,PMY2882,PMY2887,Chrom,MAF,Mdepth,MAR


In [5]:
if not 'Ref' in genos.columns:
    genos['Ref'] = [a.split('.')[0] for a in genos.Alleles]
    
if not 'Alt' in genos.columns:
    genos['Alt'] = genos['Alleles']

In [6]:
## Bring in GFF file
gffpath = '../../REF/FungiDB-46_CneoformansH99.gff.gz'
names = ["Seqid", "Source", "Type", "Start", "End", "Score", 
         "Strand", "Phase", "Attribute"]
dtype = ["str","str","str","int","int","str","str","str","str"]

gff = pd.read_csv(gffpath,comment='#',
                  sep='\t',header=None,
                  names=names,dtype=dict(zip(names,dtype)))

gff['Strand'] = gff['Strand'].replace(dict(zip(['-','+'],[-1,1])))
gff['Parent'] = [a.split('Parent=')[-1].split(';')[0].split('ID=')[-1] 
                 for a in gff.Attribute ]

gff['Gene'] = [a.split('-t26')[0] for a in gff.Parent]

gff = gff.merge(chrommap)

gff.head()

,Seqid,Source,Type,Start,End,Score,Strand,Phase,Attribute,Parent,Gene,Chrom,Length,Cumpos,Midpts
0,CP003824.1,EuPathDB,gene,409610,410982,.,1,.,ID=CNAG_01397;description=mitochondrial import...,CNAG_01397,CNAG_01397,5,1814975,6573120,7480607.5
1,CP003824.1,EuPathDB,mRNA,409610,410982,.,1,.,ID=CNAG_01397-t26_1;Parent=CNAG_01397;descript...,CNAG_01397,CNAG_01397,5,1814975,6573120,7480607.5
2,CP003824.1,EuPathDB,exon,409610,410051,.,1,.,ID=exon_CNAG_01397-E1;Parent=CNAG_01397-t26_1,CNAG_01397-t26_1,CNAG_01397,5,1814975,6573120,7480607.5
3,CP003824.1,EuPathDB,exon,410113,410259,.,1,.,ID=exon_CNAG_01397-E2;Parent=CNAG_01397-t26_1,CNAG_01397-t26_1,CNAG_01397,5,1814975,6573120,7480607.5
4,CP003824.1,EuPathDB,exon,410314,410424,.,1,.,ID=exon_CNAG_01397-E3;Parent=CNAG_01397-t26_1,CNAG_01397-t26_1,CNAG_01397,5,1814975,6573120,7480607.5


In [7]:
## Gather genes
genes = gff[(gff.Type=='gene')].sort_values(['Seqid','Start','Strand']).copy()

genes['Description'] = [a.split('description=')[-1].split('%2C')[0] 
                        for a in genes.Attribute]

genes.head()

,Seqid,Source,Type,Start,End,Score,Strand,Phase,Attribute,Parent,Gene,Chrom,Length,Cumpos,Midpts,Description
54476,CP003820.1,EuPathDB,gene,100,5645,.,-1,.,ID=CNAG_04548;description=hypothetical protein,CNAG_04548,CNAG_04548,1,2291499,0,1145749.5,hypothetical protein
59322,CP003820.1,EuPathDB,gene,5928,7982,.,-1,.,ID=CNAG_07303;description=hypothetical protein...,CNAG_07303,CNAG_07303,1,2291499,0,1145749.5,hypothetical protein
51999,CP003820.1,EuPathDB,gene,8766,9603,.,-1,.,ID=CNAG_07304;description=hypothetical protein,CNAG_07304,CNAG_07304,1,2291499,0,1145749.5,hypothetical protein
56198,CP003820.1,EuPathDB,gene,10835,11664,.,-1,.,ID=CNAG_00001;description=hypothetical protein,CNAG_00001,CNAG_00001,1,2291499,0,1145749.5,hypothetical protein
63828,CP003820.1,EuPathDB,gene,11773,12393,.,-1,.,ID=CNAG_07305;description=hypothetical protein,CNAG_07305,CNAG_07305,1,2291499,0,1145749.5,hypothetical protein


In [8]:
## Make list of descriptions and features
descriptions = ['hypothetical protein',
                'unspecified product',
                'conserved hypothetical protein']

foi = ['gene','three_prime_UTR','five_prime_UTR','CDS']

In [9]:
## Bring in reference file
refpath = '../../REF/FungiDB-46_CneoformansH99_Genome.fasta'

REF = [s for s in SeqIO.parse(refpath,format='fasta')]
len(REF)

15

In [10]:
## Collect known genes
known = genes[~(genes.Description.isin(descriptions))]
known_genes = known.Parent.tolist()
known.shape

(3950, 16)

In [11]:
## Collect unknown genes
unknown = genes[(genes.Description.isin(descriptions))]
unknown_genes = unknown.Parent.tolist()
unknown.shape

(4388, 16)

In [12]:
assert genes[~(genes.Parent.isin(unknown.Parent)) &
      ~(genes.Parent.isin(known.Parent))].shape[0] == 0

In [13]:
amoeba_qtl_8 = (8,680000,770000)
amoeba_qtl_5 = (5,1015000,1110000)
lag_37_5 = (5,1112000,1405000)


qtl = [amoeba_qtl_8,amoeba_qtl_5,lag_37_5]

In [14]:
qtl_genes = []

for (chrom,left,right) in qtl:
    qtlg = genes[(genes.Chrom==chrom) & 
                  (genes.Start<=right) & 
                  (genes.End>=left)
                ].Parent.tolist()
    
    qtl_genes.append(qtlg)
    
qtlgenes = np.unique(np.concatenate(qtl_genes))
len(qtlgenes)

219

In [15]:
## Gather parents
parents = gff[(gff.Type.isin(foi)) & 
              (gff.Type!='gene') & 
              (gff.Gene.isin(qtlgenes))
             ][['Gene','Parent']
              ].drop_duplicates()
parents.head()

,Gene,Parent
70,CNAG_01125,CNAG_01125-t26_1
397,CNAG_01082,CNAG_01082-t26_1
471,CNAG_01118,CNAG_01118-t26_1
527,CNAG_01097,CNAG_01097-t26_1
650,CNAG_01170,CNAG_01170-t26_1


In [16]:
parental_pmy = ['PMY2649', 'PMY2650']

samplename = parental_pmy[1]

In [17]:
savepath = '/home/croth/Bt22xFtc555-1/GENOTYPE/GENES/'
savepath_sample = savepath+'%s/'%samplename

if not os.path.exists(savepath):
    os.mkdir(savepath)

if not os.path.exists(savepath_sample):
    os.mkdir(savepath_sample)
    
savepath_sample

'/home/croth/Bt22xFtc555-1/GENOTYPE/GENES/PMY2650/'

In [18]:
info_cols = [c for c in genos.columns if c[:3]!='PMY']
info_cols

['Pos', 'Alleles', 'Type', 'Chrom', 'Seqid', 'Ref', 'Alt']

In [19]:
run_parents = parents.Parent.tolist()
len(run_parents)

202

In [20]:
## Is the GFF zero based?
zb = False

## Do you want to print everything 
## to the screne
## like a madman?!?!
verbos = True

In [21]:
## RUN ALL THE GENES WITH CDS
chroms = [8]#[1, 2, 3, 5, 6, 10, 11]
run_parents = gff[(gff.Type=='CDS') & (gff.Chrom.isin(chroms))].Parent.unique()
len(run_parents)

581

In [22]:
novars = []
notype = []
sofail = []
for ssk1name in run_parents:
#for ssk1name in known_genes:
    genesave = savepath_sample+ssk1name+'.csv.gz'
    
    #if os.path.exists(genesave):
    #    continue
    ## Take gene info from gff file
    ssk1df = gff[(gff.Type.isin(foi)) & 
                 (gff.Parent==ssk1name)
                ].sort_values('Start').copy()
    
    ssk1df['Start'] = ssk1df['Start'] + (1 if zb else 0)
    
    assert len(ssk1df.Strand.unique())==1
    if len(ssk1df.Type.unique())!=3:
        notype.append(ssk1name+'\n')
        continue
    ## For a sample of interest bring in data
    ##sample = pd.read_csv(samplepath,index_col=0)
    sample = genos.loc[genos[samplename].dropna().index,
                       info_cols+[samplename]]
    
    sample['Sample'] = sample[samplename]
    ## Locate genetic variants within a gene
    gv = sample[(sample.Seqid==ssk1df.Seqid.min()) & 
            (sample.Pos>=ssk1df.Start.min()) & 
            (sample.Pos<=ssk1df.End.max())]
    if gv.shape[0] == 0:
        novars.append(ssk1name+'\n')
        #if verbos:
        #    print('No variants within %s for strain %s\n'%(
        #    ssk1name,samplename))
        continue
    ## From the referecne take the sequence of interest
    gene_seq = [s.seq[ssk1df.Start.min()-(0 if zb else 1):ssk1df.End.max()] 
            for s in REF if s.id == ssk1df.Seqid.min()][0]
    ## Make a dataframe from these sequences
    refseqdf = pd.DataFrame([np.arange(ssk1df.Start.min()+(1 if zb else 0),
                                       ssk1df.End.max()+1),
                     [n for n in str(gene_seq)]],
                            index=['Pos','REF']).T;
    ## Make gene variant dataframe
    res,failed  = gvtodf(gv,refseqdf,ssk1df,verbose=verbos)
    #if (len(failed)>0) and verbos:
    #    print(ssk1name+'\n')
    #    sofail.append(ssk1name)
    res['Gene'] = ssk1name
    res['Sample'] = samplename
    ## Save and print results
    res.drop_duplicates().to_csv(genesave,index=False)
    if verbos and (len(failed)>0):
        sofail.append(genesave)
        #sofail.append(ssk1name,res.Pos.unique().shape[0],res.shape[0],
        #    res[(res.Isvar==1)].shape[0])

Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
Reference alignment failed
R

In [23]:
savepath_sample

'/home/croth/Bt22xFtc555-1/GENOTYPE/GENES/PMY2650/'

In [24]:
len(sofail)

64

In [25]:
len(novars)

117

In [31]:
failed_path = savepath_sample+'FAILED8.csv'
if (len(sofail) > 0):
    open(failed_path,'w').writelines(sofail)
    
novars_path = savepath_sample+'NOVARS8.csv'
if (len(novars) > 0):
    open(novars_path,'w').writelines(novars)

notype_path = savepath_sample+'NOTYPE8.csv'
if (len(notype) > 0):
    open(notype_path,'w').writelines(notype)

In [27]:
novars_path

'/home/croth/Bt22xFtc555-1/GENOTYPE/GENES/PMY2650/NOVARS8.csv'

In [28]:
savepath_sample

'/home/croth/Bt22xFtc555-1/GENOTYPE/GENES/PMY2650/'

In [29]:
len(run_parents) - (len(novars) + len(notype))

417